# Pourquoi et comment

**Objectif** : recueillir les pnn (càd les identifiants IdRef) des contributeurs à la revue.

**Démarche** : 
1) Avec des auteurs du BSPE : faire une estimation du nombre de noms qui manquent. 
2) 
3) 

**Documentation**: Recours à l'API SolR<a href = "http://documentation.abes.fr/aideidrefdeveloppeur/index.html"> abes </a> 


In [1]:
import re
import pandas as pd
import numpy as np 
import requests
from bs4 import BeautifulSoup
import lxml

## 1. Structurer la liste des nom/prénoms

In [2]:
with open ("../BSPE/Membres_SPE/Liste-Noms-Contributeurs.csv") as contributeurs:
    df_contributeurs = pd.read_csv(contributeurs)

In [3]:
df_contributeurs

,NOM (Prenom)
0,ACHALME (P.)
1,ADDE (R.)
2,AHMED (E.)
3,ALARY (A.)
4,ALLAIN (Jacques)
...,...
658,YVERNAULT (A.)
659,YVON (P.)
660,ZABOLOTNY
661,ZAMMIT (TH.)


In [4]:
## Séparer NOM et Prénom : 

#Nouvelle colonne pour le NOM => .iloc[i,1]
df_contributeurs["NOM"] = df_contributeurs.apply(lambda _: '', axis=1)
#Nouvelle colonne pour le PRENOM => .iloc[i,2]
df_contributeurs["Prenom"] = df_contributeurs.apply(lambda _: '', axis=1)


#Regex NOM (Prénom)
reNom = r'(^[A-Z]+)'
rePrenom = r'\((.*)\)'

for i in range(len(df_contributeurs)):
    df_contributeurs.iloc[i,1] = re.search(reNom,df_contributeurs.iloc[i,0]).group(1)
    Prenom = re.search(rePrenom,df_contributeurs.iloc[i,0])
    if Prenom:
        Prenom2=re.sub(r' {1,2}',"",Prenom.group(1))
        df_contributeurs.iloc[i,2]=Prenom2
        

In [5]:
df_contributeurs

,NOM (Prenom),NOM,Prenom
0,ACHALME (P.),ACHALME,P.
1,ADDE (R.),ADDE,R.
2,AHMED (E.),AHMED,E.
3,ALARY (A.),ALARY,A.
4,ALLAIN (Jacques),ALLAIN,Jacques
...,...,...,...
658,YVERNAULT (A.),YVERNAULT,A.
659,YVON (P.),YVON,P.
660,ZABOLOTNY,ZABOLOTNY,
661,ZAMMIT (TH.),ZAMMIT,TH.


## 2. Générer les url idref et le entrer dans le df

In [6]:
#Ajout de colonnes au df pour les url. 
df_contributeurs = df_contributeurs.reindex(columns = df_contributeurs.columns.tolist() + ['Recherche_G','Recherche_NomPrenom'])
RechercheG = []
RechercheNOMPrenom =[]

In [7]:


for i in range(len(df_contributeurs)):
#     print(df_contributeurs.iloc[i,0])
    NOM=df_contributeurs.iloc[i,1]
    Prenom=df_contributeurs.iloc[i,2]
    
    #Paramétrage : 
    ## => recordtype_z :h (càd Auteur Titre)
    
    #url générale:
    df_contributeurs.iloc[i,3]="https://www.idref.fr/Sru/Solr?q=persname_t:"+NOM+Prenom+"&sort=score%20desc&version=2.2&start=0&rows=30&indent=on&fl=id,ppn_z,recordtype_z:h,affcourt_z"
    RechercheG.append(df_contributeurs.iloc[i,3])
    
    #url NOM Prenom : 
    df_contributeurs.iloc[i,4] ="https://www.idref.fr/Sru/Solr?q=persname_t:("+NOM+"AND"+Prenom+")&sort=score%20desc&version=2.2&start=0&rows=30&indent=on&fl=id,ppn_z,recordtype_z:h,affcourt_z"
    RechercheNOMPrenom.append(df_contributeurs.iloc[i,4])
    

In [8]:
df_contributeurs

,NOM (Prenom),NOM,Prenom,Recherche_G,Recherche_NomPrenom
0,ACHALME (P.),ACHALME,P.,https://www.idref.fr/Sru/Solr?q=persname_t:ACH...,https://www.idref.fr/Sru/Solr?q=persname_t:(AC...
1,ADDE (R.),ADDE,R.,https://www.idref.fr/Sru/Solr?q=persname_t:ADD...,https://www.idref.fr/Sru/Solr?q=persname_t:(AD...
2,AHMED (E.),AHMED,E.,https://www.idref.fr/Sru/Solr?q=persname_t:AHM...,https://www.idref.fr/Sru/Solr?q=persname_t:(AH...
3,ALARY (A.),ALARY,A.,https://www.idref.fr/Sru/Solr?q=persname_t:ALA...,https://www.idref.fr/Sru/Solr?q=persname_t:(AL...
4,ALLAIN (Jacques),ALLAIN,Jacques,https://www.idref.fr/Sru/Solr?q=persname_t:ALL...,https://www.idref.fr/Sru/Solr?q=persname_t:(AL...
...,...,...,...,...,...
658,YVERNAULT (A.),YVERNAULT,A.,https://www.idref.fr/Sru/Solr?q=persname_t:YVE...,https://www.idref.fr/Sru/Solr?q=persname_t:(YV...
659,YVON (P.),YVON,P.,https://www.idref.fr/Sru/Solr?q=persname_t:YVO...,https://www.idref.fr/Sru/Solr?q=persname_t:(YV...
660,ZABOLOTNY,ZABOLOTNY,,https://www.idref.fr/Sru/Solr?q=persname_t:ZAB...,https://www.idref.fr/Sru/Solr?q=persname_t:(ZA...
661,ZAMMIT (TH.),ZAMMIT,TH.,https://www.idref.fr/Sru/Solr?q=persname_t:ZAM...,https://www.idref.fr/Sru/Solr?q=persname_t:(ZA...


## 3. Chercher s'il existe au moins une notice associée à l'url idref

Méthode : pour chaque url chercher la valeur de l'attribut numFound dans l'elt <result>
\    
`<result name="response" numFound="0" start="0"/>`

In [9]:
df_contributeurs = df_contributeurs.reindex(columns = df_contributeurs.columns.tolist() + ['Recherche_G-nbr','Recherche_NomPrenom-nbr'])    

In [10]:
df_contributeurs

,NOM (Prenom),NOM,Prenom,Recherche_G,Recherche_NomPrenom,Recherche_G-nbr,Recherche_NomPrenom-nbr
0,ACHALME (P.),ACHALME,P.,https://www.idref.fr/Sru/Solr?q=persname_t:ACH...,https://www.idref.fr/Sru/Solr?q=persname_t:(AC...,NaN,NaN
1,ADDE (R.),ADDE,R.,https://www.idref.fr/Sru/Solr?q=persname_t:ADD...,https://www.idref.fr/Sru/Solr?q=persname_t:(AD...,NaN,NaN
2,AHMED (E.),AHMED,E.,https://www.idref.fr/Sru/Solr?q=persname_t:AHM...,https://www.idref.fr/Sru/Solr?q=persname_t:(AH...,NaN,NaN
3,ALARY (A.),ALARY,A.,https://www.idref.fr/Sru/Solr?q=persname_t:ALA...,https://www.idref.fr/Sru/Solr?q=persname_t:(AL...,NaN,NaN
4,ALLAIN (Jacques),ALLAIN,Jacques,https://www.idref.fr/Sru/Solr?q=persname_t:ALL...,https://www.idref.fr/Sru/Solr?q=persname_t:(AL...,NaN,NaN
...,...,...,...,...,...,...,...
658,YVERNAULT (A.),YVERNAULT,A.,https://www.idref.fr/Sru/Solr?q=persname_t:YVE...,https://www.idref.fr/Sru/Solr?q=persname_t:(YV...,NaN,NaN
659,YVON (P.),YVON,P.,https://www.idref.fr/Sru/Solr?q=persname_t:YVO...,https://www.idref.fr/Sru/Solr?q=persname_t:(YV...,NaN,NaN
660,ZABOLOTNY,ZABOLOTNY,,https://www.idref.fr/Sru/Solr?q=persname_t:ZAB...,https://www.idref.fr/Sru/Solr?q=persname_t:(ZA...,NaN,NaN
661,ZAMMIT (TH.),ZAMMIT,TH.,https://www.idref.fr/Sru/Solr?q=persname_t:ZAM...,https://www.idref.fr/Sru/Solr?q=persname_t:(ZA...,NaN,NaN


In [18]:
def IdRef_numfound_auteur (col):
    list_numfound_IdRef = []

    for url in col:
        r = requests.get(url)
        xml = r.text
        #print(r.text)

        soup = BeautifulSoup(xml,'lxml')
        eltResult = soup.find_all('result')

        for att in eltResult:
            attnumFound = att.get("numfound")

        list_numfound_IdRef.append(attnumFound)

    return list_numfound_IdRef

#<result name="response" numFound="0" start="0"/>

In [20]:
col = df_contributeurs["Recherche_G"]
df_contributeurs["Recherche_G-nbr"] = IdRef_numfound_auteur(col)

In [21]:
df_contributeurs

,NOM (Prenom),NOM,Prenom,Recherche_G,Recherche_NomPrenom,Recherche_G-nbr,Recherche_NomPrenom-nbr
0,ACHALME (P.),ACHALME,P.,https://www.idref.fr/Sru/Solr?q=persname_t:ACH...,https://www.idref.fr/Sru/Solr?q=persname_t:(AC...,0,NaN
1,ADDE (R.),ADDE,R.,https://www.idref.fr/Sru/Solr?q=persname_t:ADD...,https://www.idref.fr/Sru/Solr?q=persname_t:(AD...,1,NaN
2,AHMED (E.),AHMED,E.,https://www.idref.fr/Sru/Solr?q=persname_t:AHM...,https://www.idref.fr/Sru/Solr?q=persname_t:(AH...,2,NaN
3,ALARY (A.),ALARY,A.,https://www.idref.fr/Sru/Solr?q=persname_t:ALA...,https://www.idref.fr/Sru/Solr?q=persname_t:(AL...,0,NaN
4,ALLAIN (Jacques),ALLAIN,Jacques,https://www.idref.fr/Sru/Solr?q=persname_t:ALL...,https://www.idref.fr/Sru/Solr?q=persname_t:(AL...,0,NaN
...,...,...,...,...,...,...,...
658,YVERNAULT (A.),YVERNAULT,A.,https://www.idref.fr/Sru/Solr?q=persname_t:YVE...,https://www.idref.fr/Sru/Solr?q=persname_t:(YV...,0,NaN
659,YVON (P.),YVON,P.,https://www.idref.fr/Sru/Solr?q=persname_t:YVO...,https://www.idref.fr/Sru/Solr?q=persname_t:(YV...,0,NaN
660,ZABOLOTNY,ZABOLOTNY,,https://www.idref.fr/Sru/Solr?q=persname_t:ZAB...,https://www.idref.fr/Sru/Solr?q=persname_t:(ZA...,2,NaN
661,ZAMMIT (TH.),ZAMMIT,TH.,https://www.idref.fr/Sru/Solr?q=persname_t:ZAM...,https://www.idref.fr/Sru/Solr?q=persname_t:(ZA...,0,NaN


In [ ]:
col = df_contributeurs["Recherche_NomPrenom"]
df_contributeurs["Recherche_NomPrenom-nbr"] = IdRef_numfound_auteur(col)